In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
from gnomad.utils.plotting import *
from typing import Set, Tuple

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201201-1619-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [21]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_table_for_RF_all_cols.ht')

In [37]:
mt = hl.read_matrix_table(
        f'{temp_dir}/ddd-elgh-ukbb/Sanger_cohorts_chr1to6-20.mt')

In [38]:
ht=mt.rows()

In [39]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 


In [40]:
ht.count()

11147182

In [33]:
allele_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_allele_data_new.ht')

In [41]:
allele_data_ht.count()

12081759

In [42]:
ht = ht.annotate(allele_data=allele_data_ht[ht.key].allele_data)

In [43]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 


In [44]:
ht.allele_data.n_alt_alleles.summarize()

Non-missing,9956141 (89.32%)
Missing,1191041 (10.68%)
Minimum,1
Maximum,1
Mean,1.00
Std Dev,0.00


In [36]:
ht=ht.n_alt_alleles.summarize()

Non-missing,9956141 (89.32%)
Missing,1191041 (10.68%)
Minimum,1
Maximum,1
Mean,1.00
Std Dev,0.00


In [25]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    'MLEAC': array<int32> 
    'MLEAF': array<float64> 
    'MQ': float64 
    'MQRankSum': float64 
    'QD': float64 
    'RAW_MQandDP': array<int32> 
    'ReadPosRankSum': float64 
    'SOR': float64 
    'info': struct {
        AC: array<int32>

In [26]:
allele_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_allele_data_new.ht')

In [27]:
allele_data_ht.summarize()

Non-missing,12081759 (100.00%)
Missing,0
Contig Counts,"{'chr5': 1453757, 'chr19': 1485, 'chr1': 2995431, 'chr20': 785942, 'chr2': 2274518, 'chr7': 1320, 'chr3': 1791365, 'chr4': 1302137, 'chr6': 1475804}"
Non-missing,12081759 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,24163518 (100.00%)
Missing,0
Min Size,1


In [29]:
ht = ht.annotate( **allele_data_ht[ht.key].allele_data)

In [30]:
ht.n_alt_alleles.summarize()

Non-missing,9950210 (89.32%)
Missing,1190095 (10.68%)
Minimum,1
Maximum,1
Mean,1.00
Std Dev,0.00


In [14]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    'MLEAC': array<int32> 
    'MLEAF': array<float64> 
    'MQ': float64 
    'MQRankSum': float64 
    'QD': float64 
    'RAW_MQandDP': array<int32> 
    'ReadPosRankSum': float64 
    'SOR': float64 
    'info': struct {
        AC: array<int32>

In [16]:
ht.allele_data.n_alt_alleles.summarize()

Non-missing,9950210 (89.32%)
Missing,1190095 (10.68%)
Minimum,1
Maximum,1
Mean,1.00
Std Dev,0.00


In [6]:
ht2.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'allele_data': struct {
        nonsplit_alleles: array<str>, 
        has_star: bool, 
        variant_type: str, 
        n_alt_alleles: int32, 
        allele_type: str, 
        was_mixed: bool
    } 
    'a_index': int32 
    'was_split': bool 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [8]:
ht2.allele_data.summarize()

Non-missing,12081759 (100.00%)
Missing,0
Non-missing,12081759 (100.00%)
Missing,0
Min Size,2
Max Size,7
Mean Size,2.25
Non-missing,27159269 (100.00%)
Missing,0
Min Size,1
Max Size,385


In [4]:
mt = hl.read_matrix_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_chr1-7and20_split.mt')

In [6]:
ht = mt.rows()
ht = ht.transmute(**ht.info)

In [7]:
trio_stats_table = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_trios_stats.ht')

In [10]:
group="raw"
trio_stats_ht = trio_stats_table.select(
        f"n_transmitted_{group}", f"ac_children_{group}"
    )

In [11]:
truth_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/truthset_table.ht')
trio_stats_table = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_trios_stats.ht')
    #inbreeding_ht = hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_inbreeding.ht')
allele_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_allele_data.ht')
allele_counts_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_qc_ac.ht')
allele_counts_ht = allele_counts_ht.select(
        *['ac_qc_samples_raw', 'ac_qc_samples_adj'])
inbreeding_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_inbreeding.ht')

In [12]:
ht = ht.annotate(
        **inbreeding_ht[ht.key],
        **trio_stats_ht[ht.key],
        **truth_data_ht[ht.key],
        **allele_data_ht[ht.key].allele_data,
        **allele_counts_ht[ht.key],
    )

In [13]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'a_index': int32 
    'was_split': bool 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    'MLEAC': array<int32> 
    'MLEAF': array<float64> 
    'MQ': float64 
    'MQRankSum': float64 
    'QD': float64 
    'RAW_MQandDP': array<int32> 
    'ReadPosRankSum': float64 
    'SOR': float64 


In [14]:
ht=ht.annotate(fail_hard_filters= (ht.QD < 2) | (ht.FS > 60) | (ht.MQ < 30))

In [15]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'a_index': int32 
    'was_split': bool 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    'MLEAC': array<int32> 
    'MLEAF': array<float64> 
    'MQ': float64 
    'MQRankSum': float64 
    'QD': float64 
    'RAW_MQandDP': array<int32> 
    'ReadPosRankSum': float64 
    'SOR': float64 


In [16]:
ht = ht.checkpoint(
        f'{tmp_dir}/ddd-elgh-ukbb/Sanger_table_for_RF_all_columns.ht', overwrite=True)

KeyboardInterrupt: 

2020-11-26 11:22:55 Hail: INFO: wrote table with 13651809 rows in 10000 partitions to hdfs://spark-master:9820//ddd-elgh-ukbb/Sanger_table_for_RF_all_columns.ht
